<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_NSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 62.9 MB/s 
     |████████████████████████████████| 6.5 MB 46.3 MB/s 
     |████████████████████████████████| 895 kB 66.6 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Softmax
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [3]:
# 모델과 tokenizer 준비

model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

# 동일 문장 여부 파악

In [7]:
# 임의의 문장 예측 (네이버 기사)

# 이어지는 문장
prompt1 = '정부는 약사 등의 '
next_sentence1 = '환자 발생으로 약국 영업이 중단될 경우 해당 지역의 보건 서비스에 심각한 영향이 우려돼 이번 지침을 마련했다고 설명했다.'

# 이어지지 않는 문장
prompt2 = '튜브 겉면에도 복합 재질을 의미하는 '
next_sentence2 = '연기를 할 수 없어 이같은 방법을 선택한 것으로 보인다.'

In [10]:
def same_sentence(prompt, next):
  encoding = tokenizer(prompt, next, return_tensors='tf')
  logits = model(encoding['input_ids'], token_type_ids = encoding['token_type_ids'])[0]
  softmax = Softmax()
  probs = softmax(logits)
  res = tf.math.argmax(probs, axis=-1).numpy()

  return res

In [11]:
res1 = same_sentence(prompt1, next_sentence1)

if res1 == 0:
  print('두 문장은 이어지는 문장입니다.')
else:
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지는 문장입니다.


In [12]:
res2 = same_sentence(prompt2, next_sentence2)

if res2 == 0:
  print('두 문장은 이어지는 문장입니다.')
else: 
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지지 않는 문장입니다.


# 데이터셋 준비

In [16]:
import os

root = '/content/drive/MyDrive/news_class9x1400/'
categories = os.listdir(root)

dataset = []

for category in categories:
  files = os.listdir(root + category)
  for i, f in enumerate(files):
    fname = root + category + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read()
    dataset.append([strings])
    file.close()
    if len(dataset) > 5:
      break
  

In [18]:
table = str.maketrans({
    '\'': '',
    '\"': '',
    '★': '',
    '▲': '',
    '┌': '',
    '├': '',
    '│': '',
    '└': '',
    '→': ''

})

In [19]:
process_dataset = []

for sentences in dataset:
  sentences = sentences[0]
  sentences = sentences.translate(table)
  sentences = sentences.split('\n')
  process_dataset.extend(sentences)

process_dataset

['기타 신동도 이 대통령도 유튜브로 돈 벌 수 있다',
 '한국 개인 사용자들이 전 세계를 상대로 수익을 얻을 기회다.',
 '온라인 동영상 사이트 유튜브는 6일 오전 10시 30분 서울 역삼동 구글코리아 본사에서 유튜브 3.0을 소개하는 기자간담회를 열었다.',
 '유튜브는 하루 조회수 30억 건에 이르는 세계 최대 온라인 동영상 사이트로 한국을 포함해 전세계 32개국에서 서비스하고 있다.',
 '구글 검색, 지메일 등 다른 서비스들이 네이버, 다음과 같은 토종 포털에 밀려 한국 시장에서 고전하고 있는 와중에도 유튜브는 국내 동영상 사이트 1위를 고수해 왔다.',
 '유튜브는 이날 2008년 1월 한국 서비스 당시 단순히 사용자들이 동영상을 올리고 나누는 개념이었던 1.0세대, SM, JYP, YG 등 국내 파트너사들과 수익을 배분하는 2.0 세대에 이어 개인 사용자들과도 광고 수익을 나누는 3.0 세대 개념을 발표했다.',
 '일본 도쿄에서 화상으로 연결된 아담 스미스 유튜브 아시아태평양지역 총괄은 유튜브는 이제 스마트폰, 태블릿, TV 등을 통해 언제, 어디서, 어떤 기기로든 시청할 수 있다면서 창의적 사용자들이 동영상에 광고를 붙여 수익화할 수 있게 유튜브가 도와줄 것이라고 밝혔다.',
 '스미스 총괄은 기타 연주 동영상을 유튜브에 올려 3억 회 이상 조회수를 기록한 14살 기타 신동 정성하군 사례를 들며 파트너를 일반 사용자까지 확대하겠다고 밝혔다.',
 '그동안 동영상 저작권 침해 문제로 콘텐츠 사업자들과 크고 작은 마찰을 빚었던 유튜브는 저작권 보호 솔루션인 콘텐츠 ID 기술을 도입해 세계 미디어 기업과 파트너 관계를 맺고 있다.',
 '콘텐츠 ID는 파트너사에서 동영상을 올린 뒤 일반 사용자들이 같은 동영상을 올릴 경우 이를 가려내 파트너 스스로 저작권을 관리할 수 있게 하는 기술이다.',
 '스미스는 콘텐츠ID 덕분에 파트너사 저작권을 보호하고 수익 창출이 가능해 세계 굴지 미디어 기업과 파트너를 맺을 수 있다, 아시아에선 한국 K팝과 한류가

In [25]:
# 너무 짧은 문장은 제거

for sent in process_dataset:
  # if len(sent) <= 5:
  #   process_dataset.remove(sent)
  if len(sent) <= 10:
    process_dataset.remove(sent)

In [33]:
import pandas as pd

split_sentence = pd.DataFrame(columns=['prompt', 'next', 'same'])
split_sentence.head()

,prompt,next,same


In [34]:
from random import randint

# 문장 분리

index = 0

for sent in process_dataset:
  x = randint(0, len(sent)-1)
  split_sentence.loc[index] = [sent[:x], sent[x:], 1]
  index += 1

split_sentence

,prompt,next,same
0,기타 신동,도 이 대통령도 유튜브로 돈 벌 수 있다,1
1,한국 개인,사용자들이 전 세계를 상대로 수익을 얻을 기회다.,1
2,온라인 동영상 사이트 유튜브는 6일 오전 10시 30분 서울 역,삼동 구글코리아 본사에서 유튜브 3.0을 소개하는 기자간담회를 열었다.,1
3,,유튜브는 하루 조회수 30억 건에 이르는 세계 최대 온라인 동영상 사이트로 한국을 ...,1
4,"구글 검색, 지메일 등 다른 서비스들이 네이버, 다음과 같은 토종 포털에 밀려 한국...",1위를 고수해 왔다.,1
...,...,...,...
520,"그러면서 그는 국회에서 중소영세상인 살리기 의원 모임을 만들기로 했고, 중소상인과 ...",완벽한 제도가 되도록 힘을 모아야 할 것이라고 제시했다.,1
521,동네구멍가게는 대기업들이 하는 장사,아니다,1
522,이휘웅 경남유통상인연합회 회장은 인사말을 통해 대기업은 대기업답게 해야 될 사업이 ...,"장사로, 우리나라를 대표하는 대기업들이 하는 장사가 아니다고 말했다.",1
523,"손석형 경남도의원은 세계화, 글로벌 등으로 외국문화와 자본을 무조건 받아들이다 보니...","기 위한 준비와 정책, 지원이 필요한데 정부는 손을 놓고 있다고 비판했다.",1
